<br>

# Introdução

In [ ]:
import os
import math
import time
import shutil
import requests
import pandas as pd
from datetime import date, datetime
from bs4 import BeautifulSoup
from dateutil.relativedelta import relativedelta

In [ ]:
import sys
sys.path.append(os.path.expanduser('~/Documents/Sourcecode'))
from my_vault.credentials_infoaguas import credential

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

<br>

# Functions

<br>

## Cria Driver

In [ ]:
def create_driver(download_path, headless=True):
    # Xpath
    url = 'https://addons.mozilla.org/firefox/downloads/file/3588871/xpath_finder-1.0.2-fx.xpi'
    r = requests.get(url)
    adds_path = os.path.join('..', 'adds')
    os.makedirs(adds_path, exist_ok=True)
    with open(os.path.join(adds_path, 'xpath.xpi'), 'wb') as f:
        f.write(r.content)

    # Create directory
    os.makedirs(download_path, exist_ok=True)
    
    # Driver Firefox com Profile
    profile = webdriver.FirefoxProfile()
    profile.set_preference('intl.accept_languages', 'pt-BR, pt')
    profile.set_preference('browser.download.folderList', 2)
    profile.set_preference('browser.download.dir', download_path)
    profile.set_preference('browser.download.manager.showWhenStarting', 'false')
    profile.set_preference('browser.helperApps.neverAsk.saveToDisk', 'application/octet-stream;application/vnd.ms-excel;text/html')
    profile.update_preferences()
    
    # Driver Firefox com Options
    options = Options()
    options.headless = headless

    # Driver
    log_path = os.path.join('..', 'logs')
    os.makedirs(log_path, exist_ok=True)
    driver = webdriver.Firefox(
        firefox_profile=profile,
        options=options,
        service_log_path=os.path.join(log_path, 'geckodriver.log')
    )
    driver.install_addon(os.path.abspath(os.path.join(adds_path, 'xpath.xpi')), temporary=True)
    return driver

<br>

## Go Login

In [ ]:
def go_login(payload):
    url = 'https://sistemainfoaguas.cetesb.sp.gov.br/'
    driver.get(url)
    time.sleep(2)
    #driver.implicitly_wait(10)
    
    try:
        # Login
        username = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, "//*[@id='Email']")))
        password = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, "//*[@id='Senha']")))
        time.sleep(2)
        
        username.send_keys(payload['username'])
        password.send_keys(payload['password'])
        time.sleep(1)
        driver.find_element_by_xpath("//button[@onclick=\"entrarClick();\"]").click()
        time.sleep(3)
    except:
        print('Talvez já estivesse logado!?')

<br>

## Go Relatório Superficial

In [ ]:
def go_rel_superficiais():
    # Relatório de Águas Superficiais
    url = 'https://sistemainfoaguas.cetesb.sp.gov.br/AguasSuperficiais/RelatorioQualidadeAguasSuperficiais'
    driver.get(url)
    #driver.find_element_by_xpath("//button[@type=\"submit\"]").click()
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, "//button[@type=\"submit\"]"))).click()

<br>

## Get Table

Monta Tabela com Todos os Pontos

In [ ]:
def get_df():
    # Vai para a primeira página!
    WebDriverWait(driver, 30).until(EC.presence_of_element_located(
        (By.XPATH, "//*[@data-dt-idx='1']"))).click()
    
    # Número de Páginas
    WebDriverWait(driver, 30).until(EC.presence_of_element_located(
        (By.XPATH, "//*[@name='gridConsulta_length']//option[text()='100']"))).click()
    info = driver.find_element_by_xpath("//*[@id='gridConsulta_info']").text
    n_pages = math.ceil(int(info.split(' ')[-2])/100)

    dfs = []
    for i in range(0, n_pages):
        time.sleep(0.5)
        driver.execute_script("window.scrollTo(0, 0)")
        soup = BeautifulSoup(driver.page_source)
        soup_table = soup.find('table')
        df = pd.read_html(str(soup_table))
        df = df[0]
        dfs.append(df)
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') # Rola para baixo
        driver.find_element_by_xpath("//*[@id='gridConsulta_next']").click()
        time.sleep(0.5)

    # Concatena
    df = pd.concat(dfs)
    df['Data Início'] = pd.to_datetime(df['Data Início'], format='%d/%m/%Y', errors='coerce')
    df['Data Fim'] = pd.to_datetime(df['Data Fim'], format='%d/%m/%Y', errors='coerce')
    df.drop(['Unnamed: 6'], axis=1, errors='ignore', inplace=True)
    df.sort_values(by=['Cód.Ponto'], ascending=True, inplace=True)
    df.reset_index(drop=True, inplace=True)
    print('Tabela com "{}" pontos obtida'.format(len(df)))
    return df

<br>

## Set Ponto

Função que pesquisa um ponto

In [ ]:
def set_ponto(ponto):
    # Clica em Consulta
    WebDriverWait(driver, 30).until(EC.presence_of_element_located(
        (By.XPATH, "//*[@aria-controls='gridConsulta']")))
    
    # Procura Ponto
    search = WebDriverWait(driver, 30).until(EC.presence_of_element_located(
        (By.XPATH, "//*[@aria-controls='gridConsulta']")))
    time.sleep(0.5)
    search.clear()
    search.send_keys(ponto)
    
    # Pesquisa
    WebDriverWait(driver, 30).until(EC.presence_of_element_located(
        (By.XPATH, "//*[@id='CodigoPonto']"))).click()
    
    time.sleep(1)
    print('> Ponto "{}" definido'.format(ponto))
    time.sleep(1)

<br>

## Set Dates

In [ ]:
def set_date(tipo_data, data):
    # Checa se o botão "Gerar Planilha" está disponível!
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, "//*[@class='btn btn-primary']")))
    
    # Dictionary Months
    meses = {
        1: 'Jan',
        2: 'Fev',
        3: 'Mar',
        4: 'Abr',
        5: 'Mai',
        6: 'Jun',
        7: 'Jul',
        8: 'Ago',
        9: 'Set',
        10: 'Out',
        11: 'Nov',
        12: 'Dez',    
    }
    # Data
    time.sleep(2)
    driver.find_element_by_xpath("//*[@id='" + tipo_data + "']").click() #tipo_data = 'DataInicial' ou 'DataFinal'
    driver.find_element_by_xpath("//*[@class='datepicker']//*[@class='datepicker-days']//*[@class='picker-switch']").click()
    # WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, "//*[@id='" + tipo_data + "']")))
    # WebDriverWait(driver, 10).until(EC.presence_of_element_located(
    #     (By.XPATH, "//*[@class='datepicker']//*[@class='datepicker-days']//*[@class='picker-switch']"))).click()
    
    # Get Year
    year = driver.find_element_by_xpath(
        "//*[@class='datepicker']//*[@class='datepicker-months']//*[@class='picker-switch']")
    year_n = int(year.text)
    while year.text != str(data.year):        
        prev_year = driver.find_element_by_xpath(
            "//*[@class='datepicker']//*[@class='datepicker-months']//*[@class='prev']").click()
        year = driver.find_element_by_xpath(
            "//*[@class='datepicker']//*[@class='datepicker-months']//*[contains(@class, 'picker-switch')]")
        year_n = int(year.text)

    # Get Month
    months = driver.find_elements_by_xpath(
        "//*[@class='datepicker']//*[@class='datepicker-months']//*[contains(@class, 'month')]")
    for month in months:
        if month.text == meses[data.month]:
            month_n = int(list(meses.keys())[list(meses.values()).index(month.text)])
            month.click()
            break

    # Get Days
    days = driver.find_elements_by_xpath(
        "//*[@class='datepicker-days']//*[contains(@class, 'day')]")
    for day in days:
        if day.text == str(data.day):
            day_n = int(day.text)
            day.click()
            break
    
    # Message
    if date(year_n, month_n, day_n) == data:
        print('> {} {} definida'.format(
            tipo_data.replace('Data', 'Data '),
            data.strftime('%d.%m.%Y')))
    else:
        print('> Deu ruim!')
    
    time.sleep(1)
    return

In [ ]:
def set_both_dates(startdate, enddate):
    set_date('DataInicial', startdate)
    set_date('DataFinal', enddate)

<br>

## Set Download

In [ ]:
def set_download_file(startdate, enddate):
    # Clica para Gerar Planilha
    time.sleep(1)
    WebDriverWait(driver, 30).until(EC.presence_of_element_located(
        (By.XPATH, "//*[@class='btn btn-primary']"))).click()
    time.sleep(5)
    
    # Procura mensagem
    # try:
    #     msg = driver.find_element_by_xpath("//*[@class='field-validation-error alert alert-danger']").text
    # except:
    #     msg = ''
    list_errors = driver.find_elements_by_xpath('//*[contains(@class,"field-validation-error") or @id="erroPeriodo"]')
    msg = ''
    for error in list_errors:
        msg = error.text
    
        if msg == 'Não há informações para o ponto e período selecionados':
            # Clica em Voltar
            driver.find_element_by_xpath("//*[@class='btn btn-default']").click()
            print(
                '> {}: {} - {}'.format(
                    msg,
                    startdate.strftime('%d.%m.%Y'),
                    enddate.strftime('%d.%m.%Y')
                )
            )
            pass
        if msg == 'O período selecionado não poderá ter um intervalo superior a 5 anos' or msg == 'O campo Data Inicial deve ser menor que o campo Data Final':
            print(
                '> {}: {} - {}'.format(
                    msg,
                    startdate.strftime('%d.%m.%Y'),
                    enddate.strftime('%d.%m.%Y')
                )
            )
            raise Exception('Ajustar função "set downloads"')

        # Faz o download e renomeia
        if msg == '':
            # Artificio para ver se já voltou para a opção da consulta
            WebDriverWait(driver, 30).until(EC.presence_of_element_located(
                (By.XPATH, "//*[@aria-controls='gridConsulta']")))
    print('> Clicou em Download')

In [ ]:
try:
    ponto = 'BALD02600'
    set_ponto(ponto)

    startdate = date(2016, 1, 1)
    enddate = date(2015,2,1)

    set_both_dates(startdate, enddate)

    set_download_file(startdate, enddate)
except:
    pass

<br>

## Rename File

In [ ]:
def rename_file(directory, ponto, startdate, enddate):
    # Cria Diretório
    os.makedirs(directory, exist_ok=True)
    
    # Define o novo nome
    filename_new = '{} {} {}.xlsx'.format(
        ponto,
        startdate.strftime('%Y-%m-%d'),
        enddate.strftime('%Y-%m-%d')
    )    

    # Avalia o conteúdo do diretório! Vê quantos arquivos de interesse tempos!
    # Vê se começa com "RelatorioQualidadeAguasSuperficiais" e termina com ".xlsx". Não será aceito "part.xlsx"!
    list_files = os.listdir(os.path.join(directory))
    list_files_rel = [i for i in list_files if i.startswith('RelatorioQualidadeAguasSuperficiais') and i.split('.', 1)[-1] == 'xlsx']
   
    if len(list_files_rel) == 0:
        pass
    if len(list_files_rel) == 1:
        print('> Download Ok!')
        
        # Se tem o arquivo
        # TODO: Add *.part ou *.crdownload
        teste = os.path.isfile(os.path.join(directory, 'RelatorioQualidadeAguasSuperficiais.xlsx'))
        while teste:
            shutil.move(
                max([os.path.join(directory, f) for f in os.listdir(directory)], key=os.path.getctime),
                os.path.join(directory, filename_new)
            )
            time.sleep(2)
            teste = os.path.isfile(os.path.join(directory, 'RelatorioQualidadeAguasSuperficiais.xlsx'))
            time.sleep(2)
        print('> Arquivo renomeado para "{}"'.format(filename_new))
        
    if len(list_files_rel) > 1:
        print('Tem mais de arquivo que inicia com "RelatorioQualidadeAguasSuperficiais"!\nno diretório "{}"\nErro!!!!'.format(directory))
        print(directory)
        print(list_files_rel)
        raise Exception('Ajustar função "rename file"')
    time.sleep(2)

<br>

## Create df *download*

In [ ]:
def inverval(x):
    years = relativedelta(x['enddate_fill'], x['stardate_fill']).years
    months = relativedelta(x['enddate_fill'], x['stardate_fill']).months
    days = relativedelta(x['enddate_fill'], x['stardate_fill']).days
    #print('{} anos - {} meses- {} dias'.format(years, months, days))
    return years + months/12 + days/500

In [ ]:
def create_df_download(df):
    # Blank List
    list_dates = []
    
    # Anos que pode fazer download
    limit_years = 5
    
    # Set Date Min
    startdate_global = min(df['Data Início'])
    enddate_global = date.today()
    
    # Fill
    df['stardate_fill'] = df['Data Início'].fillna(startdate_global)
    df['stardate_fill'] = df['stardate_fill'].apply(lambda x: date(x.year, 1, 1))
    
    # Adjust Data
    df['enddate_fill'] = df['Data Fim'].fillna(enddate_global)
    df['enddate_fill']  = df['enddate_fill'].apply(lambda x: date(x.year, 12, 31))
    df['enddate_fill'] = df['enddate_fill'].apply(lambda x: x if x < date.today() else date.today())
    
    # 
    df['n_years'] = df.apply(lambda x: inverval(x), axis=1)
    df['n_files'] = df['n_years'].apply(lambda x: math.ceil(x/limit_years))
    
    # 
    display(df.head())

    # Para Cada Ponto (dentro da tabela com lista de pontos)
    for i, row in df.iterrows():
        # Parameters
        ponto_mon = row['Cód.Ponto']
        startdate = row['stardate_fill']
        enddate = row['enddate_fill']
        n_years = row['n_years']
        n_files = row['n_files']
        
        # Loop        
        for file in range(0, n_files):
            # Define uma Data Final com avanço de X anos
            enddate_loop = date((startdate + relativedelta(years=limit_years)).year - 1, 12, 31)
            enddate_loop = enddate if enddate_loop > enddate else enddate_loop
            
            # Append
            dict_input = {
                'Cód.Ponto': ponto_mon,
                'Data Início': startdate,
                'Data Fim': enddate_loop
            }
            list_dates.append(dict_input)
            
            # Define uma Data Inicial com avanço de X anos
            startdate = startdate + relativedelta(years=limit_years)            
    return pd.DataFrame(list_dates)

In [ ]:
# Read Dataframe
df = pd.read_csv(
    os.path.join('..', 'data', 'tab_pontos_monitoramento.csv'),
    parse_dates=['Data Início', 'Data Fim'],
)

# Results
cod = 'ABRA02950'
cod = 'BALD02600'

print('Tabela com "{}" pontos obtida'.format(len(df)))
display(df.head())
df = df[df['Cód.Ponto'] == cod]

try:
    df_down = create_df_download(df)
    df_down = df_down[df_down['Cód.Ponto'] == cod]
    display(df_down.head())
except:
    pass

<br>

# Run

<br>

## Pontos de Monitoramento

In [ ]:
# Driver
download_path = os.path.join(os.getcwd(), '..')
driver = create_driver(download_path, headless=True)

# Faz o login e vai para Relatório de Águas Superficiais
go_login(credential)
go_rel_superficiais()

# Pega toda a tabela
df = get_df()
df.head()

# Salva Tabela
input_path = os.path.join('..', 'data')
os.makedirs(input_path, exist_ok=True)
df.to_csv(
    os.path.join(input_path, 'tab_pontos_monitoramento.csv'),
    index=False,
)

# Close
driver.quit()

<br>

## *Scrapy* um ponto

In [ ]:
# Cria driver com 'download path' definido
download_path = os.path.join(os.getcwd(), '..', 'data', 'individuais', 'D')
driver = create_driver(download_path, headless=True)

# Faz o login e vai para Relatório de Águas Superficiais
go_login(credential)
go_rel_superficiais()

In [ ]:
# Escolhe um Ponto de Monitoramento
cod_ponto = 'DADO02600' # Deu erro em ABCA02300
set_ponto(cod_ponto)

# Escolhe uma data de Início e Fim
startdate = date(2007, 1, 1)
enddate = date(2011, 12, 31)
set_both_dates(startdate, enddate)

# Faz Download
set_download_file(startdate, enddate)
rename_file(download_path, cod_ponto, startdate, enddate)

In [ ]:
driver.quit()

<br>

## *Scrapy* vários pontos

In [ ]:
# Read Dataframe
df = pd.read_csv(
    os.path.join('..', 'data', 'tab_pontos_monitoramento.csv'),
    parse_dates=['Data Início', 'Data Fim'],
)

# Results
print('Tabela com "{}" pontos obtida'.format(len(df)))
display(df.head())

# Create Dataframe to download
df_down = create_df_download(df)
print('Serão "{}" arquivos para baixar'.format(len(df_down)))
display(df_down.head())

# Salva Tabela
input_path = os.path.join('..', 'data')
os.makedirs(input_path, exist_ok=True)
df_down.to_csv(
    os.path.join(input_path, 'tab_pontos_monitoramento_download.csv'),
    index=False,
)

# Read Dataframe
df = pd.read_csv(
    os.path.join(input_path, 'tab_pontos_monitoramento_download.csv'),
    parse_dates=['Data Início', 'Data Fim'],
)

# FILTROS 
# Seleciona pontos exceto determinado ponto da lista!
#df = df[df['Cód.Ponto']!='SDOM04600']

# Seleciona pontos que começam com determinado conjunto de letras
df = df[df['Cód.Ponto'].str.startswith('PARN02900')]

# Seleciona pontos após um ponto específico
#pto_monitoramento = 'JAHU02500'
#df = df[df.index >= df.index[df['Cód.Ponto'] == pto_monitoramento].values[0]]

# RESULTS
print('Serão "{}" arquivos para selecionados'.format(len(df)))
display(df.head())

In [ ]:
# Loop
for i, row in df.iterrows():
    print('\n> Loopar "{}"'.format(row['Cód.Ponto']))
    # Enquanto não termina download com sucesso, não continua
    download_ok = None
    while download_ok is None:
        try:
            # Parameters
            cod_ponto = row['Cód.Ponto']
            startdate = row['Data Início']
            enddate = row['Data Fim']

            # Seleciona o Ponto
            set_ponto(cod_ponto)

            # Definir Datas
            set_both_dates(startdate, enddate)

            # Gerar Planilha
            set_download_file(startdate, enddate)
            rename_file(download_path, cod_ponto, startdate, enddate)

            # Anotações
            download_ok = True

        except:
            # Close
            try: driver.quit()
            except: pass
            
            # Cria Driver
            download_path = os.path.join(os.getcwd(), '..', 'data', 'individuais')
            driver = create_driver(download_path, headless=True)

            # Faz login
            go_login(credential)
            go_rel_superficiais()

            # Deleta arquivos problemáticos e não renomeados
            list_files = os.listdir(os.path.join(download_path))
            [os.remove(os.path.join(download_path, f)) for f in list_files if f.startswith('RelatorioQualidadeAguasSuperficiais')]
            
            # Close
            print('{} >>> Reiniciando. Deu problemas.'.format(datetime.today()))
    
# Finished
try: driver.quit()
except: pass
print('\n{} >>> FIM!'.format(datetime.today()))

In [ ]:
# https://stackoverflow.com/questions/56085152/selenium-python-error-element-could-not-be-scrolled-into-view